<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
<p>
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 
</p>    
<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.
<p>
Le modèle étudié se base sur les données fournies par le site de StackOverFlow.
</p>
<p>
Ce notebook présente un système de suggestion de TAGS basé sur des méthodes statistiques.
</p>
<p>
Le traitement des données réalisé dans le notebook 'P6_DataAnalysis' sont utilisées pour alimenter les modèles.
</p>



# 5 Statistical TAGS assignement methods

TAGS are regarded as keywords.

Aims of this section is to create data points then identify clusters from this set of data.

These data-points are digital representation of the cleaned documents dataset.

For each-one of the clusters, most representative terms will be identified.

These terms will be considered as TAGs.

Differents embeddings techniques will be used in order to create data-points : 

* Bag of words
* Co-occurence
* TF-IDF

**Loading cleaned train dataset.**

In [18]:
import p5_util
file_name='./data/df_sof_train.dump'
df_sof_train = p5_util.object_load(file_name)
print(df_sof_train.shape)

p5_util.object_load : fileName= ./data/df_sof_train.dump
(24604, 5)


### 5.1.1 Features creation from Bag of Words

* Dataset is tokenized
* Corpus is represented as a matrix
* Matrix values are tokens frequencies (TF)

In [19]:
dict_sof_document = df_sof_train.Body.to_dict()

**Normalization**

In [20]:
# When building vacabulary, terms with frequency document < p_min_df are ignored.
p_min_df = 0.001

# When building vacabulary, terms with frequency document> p_max_df are ignored.
p_max_df = 1.

**BOW Vectorization with (1-GRAM terms)**

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
ngram=1
vectorizer = CountVectorizer(min_df=0.001, max_df=.1, ngram_range=(ngram, ngram))
csr_matrix = vectorizer.fit_transform(dict_sof_document.values())

print(csr_matrix.shape)

(24604, 2147)


The 24604 posts are expanded over 2462 features.

**Test of statistical methods for TAGs assignation**

For each raw from CSR matrix, the N greater values are extracted.

Matching Key words from columns are chosen from vacabulary.

Those keywords are assigned as TAGS.


**Loading test dataset**

In [22]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)

**Extract a row to be tested from test dataset**

In [23]:
import p6_util

document_index=400

In [ ]:
import p6_util

df_document = df_sof_test[df_sof_test.index==document_index]
df_document_std = p6_util.p6_df_standardization(df_document)

**Apply test**

In [24]:
import p6_util
p_tag_ratio=0.1
embeding_mode='bow'
list_tag, list_tag_original, str_original_document \
= p6_util.taglist_stat_predict(df_sof_test, document_index, embeding_mode\
                               , vectorizer, csr_matrix, p_tag_ratio)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeroc words from sentences...

Removing stopwords...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_marker,args=('code',))
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_clean_lxml)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .


Lemmatization ...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_stopwords)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Body']=df.Body.apply(p5_util.cb_lemmatizer,args=(lemmatizer,'lower'))


In [25]:
if list_tag is not None and list_tag_original is not None and str_original_document is not None: 
    print("\nOriginal sentence : {}".format(str_original_document))
    print("BOW : Suggested Tags list : {}".format(sorted(list_tag)))
    print("Original Tags list : {}\n".format(sorted(list_tag_original)))


Original sentence : <p>In my user model, I have an attribute called "nickname" and validates as such:</p>

<blockquote>
  <p>validates_format_of :nickname, :with => /[a-zA-Z0-9]$/, :allow_nil => true</p>
</blockquote>

<p>However, it is currently letting this string pass as valid:</p>

<p>a?c</p>

<p>I only want to accept alphanumeric strings - does anyone know why my regular expression is failing?  If anybody could suggest a better regular expression, I'm all ears.</p>

BOW : Suggested Tags list : ['allow', 'anybody']
Original Tags list : ['regex', 'validation']



### 5.1.2 Features creation from TF-IDF representation

* Terms having a frequency occurence greater then ``p_max_df`` threshold are ignored
* Terms having a frequency occurence less then ``p_min_df`` threshold are ignored

**Normalization**

In [26]:
# When building vacabulary, terms with frequency document < p_min_df are ignored.
p_min_df = 0.001

# When building vacabulary, terms with frequency document> p_max_df are ignored.
p_max_df = 1.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

dict_sof = df_sof_train.Body.to_dict()
print(len(dict_sof))

vectorizer=TfidfVectorizer(norm="l2", use_idf=True, min_df=p_min_df, max_df=p_max_df)
csr_matrix = vectorizer.fit_transform(dict_sof.values())

print(csr_matrix.shape)

24604
(24604, 2166)


The 24604 posts are expanded over 2484 features.

**Test of statistical methods based on TFIDF for TAGs assignation**

For each raw from CSR matrix, the N greater values are extracted.

Matching Key words with columns are chosen from vacabulary.

Those keywords are assigned as TAGS.


**Loading test dataset**

In [28]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)

**Extract a row to be tested from test dataset**

In [29]:
import p6_util

document_index=400

**Apply test**

In [30]:
import p6_util
p_tag_ratio=0.3
embeding_mode='tfidf'
list_tag, list_tag_original, str_original_document \
= p6_util.taglist_stat_predict(df_sof_test, document_index, embeding_mode, vectorizer, csr_matrix, p_tag_ratio)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeroc words from sentences...

Removing stopwords...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_marker,args=('code',))
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_clean_lxml)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .


Lemmatization ...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_stopwords)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Body']=df.Body.apply(p5_util.cb_lemmatizer,args=(lemmatizer,'lower'))


In [31]:
if list_tag is not None and list_tag_original is not None and str_original_document is not None: 
    print("\nOriginal sentence : {}".format(str_original_document))
    print("TFIDF : Suggested Tags list : {}".format(list_tag))
    print("Original Tags list : {}\n".format(list_tag_original))


Original sentence : <p>In my user model, I have an attribute called "nickname" and validates as such:</p>

<blockquote>
  <p>validates_format_of :nickname, :with => /[a-zA-Z0-9]$/, :allow_nil => true</p>
</blockquote>

<p>However, it is currently letting this string pass as valid:</p>

<p>a?c</p>

<p>I only want to accept alphanumeric strings - does anyone know why my regular expression is failing?  If anybody could suggest a better regular expression, I'm all ears.</p>

TFIDF : Suggested Tags list : ['allow', 'anybody', 'attribute', 'ci', 'expression', 'pas', 'regular', 'valid']
Original Tags list : ['regex', 'validation']



### 5.1.3 Features creation from Co-occurency representation : bi-gram occurencies

* Terms having a frequency occurence greater then ``p_max_df`` threshold are ignored
* Terms having a frequency occurence less then ``p_min_df`` threshold are ignored

In [32]:
# When building vacabulary, terms with frequency document < p_min_df are ignored.
p_min_df = 0.001

# When building vacabulary, terms with frequency document> p_max_df are ignored.
p_max_df = 1.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
ngram=2
vectorizer = CountVectorizer(min_df=0.001, max_df=.1, ngram_range=(ngram, ngram))
csr_matrix = vectorizer.fit_transform(dict_sof_document.values())

print(csr_matrix.shape)

(24604, 1272)


The 24604 posts are expanded over 1376 features.

**Apply test **

In [34]:
import p6_util
p_tag_ratio=0.3
embeding_mode='ngram'
list_tag, list_tag_original, str_original_document \
= p6_util.taglist_stat_predict(df_sof_test, document_index, embeding_mode, vectorizer, csr_matrix, p_tag_ratio)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_marker,args=('code',))
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_clean_lxml)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .


Filtering alpha-numeroc words from sentences...

Removing stopwords...

Lemmatization ...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_stopwords)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Body']=df.Body.apply(p5_util.cb_lemmatizer,args=(lemmatizer,'lower'))


In [35]:
if list_tag is not None and list_tag_original is not None and str_original_document is not None: 
    print("\nOriginal sentence : {}".format(str_original_document))
    print("BOW/NGRAM=2 : Suggested Tags list : {}".format(list_tag))
    print("Original Tags list : {}\n".format(list_tag_original))


Original sentence : <p>In my user model, I have an attribute called "nickname" and validates as such:</p>

<blockquote>
  <p>validates_format_of :nickname, :with => /[a-zA-Z0-9]$/, :allow_nil => true</p>
</blockquote>

<p>However, it is currently letting this string pass as valid:</p>

<p>a?c</p>

<p>I only want to accept alphanumeric strings - does anyone know why my regular expression is failing?  If anybody could suggest a better regular expression, I'm all ears.</p>

BOW/NGRAM=2 : Suggested Tags list : ['regular expression']
Original Tags list : ['regex', 'validation']



### 5.1.4 Features creation from Co-occurency representation : 1-gram & bi-gram TF-IDF

* Terms having a frequency occurence greater then ``p_max_df`` threshold are ignored
* Terms having a frequency occurence less then ``p_min_df`` threshold are ignored

In [36]:
# When building vacabulary, terms with frequency document < p_min_df are ignored.
p_min_df = 0.001

# When building vacabulary, terms with frequency document> p_max_df are ignored.
p_max_df = 1.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

ngram1=1
ngram2=2
vectorizer=TfidfVectorizer(norm="l2", use_idf=True, min_df=p_min_df, max_df=p_max_df, ngram_range=(ngram1, ngram2))

csr_matrix = vectorizer.fit_transform(dict_sof_document.values())

print(csr_matrix.shape)

(24604, 3438)


The 24604 posts are expanded over 3860 features.

**Apply test**

In [38]:
import p6_util
p_tag_ratio=0.3
embeding_mode='tfidf'
list_tag, list_tag_original, str_original_document \
= p6_util.taglist_stat_predict(df_sof_test, document_index, embeding_mode, vectorizer, csr_matrix, p_tag_ratio)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeroc words from sentences...

Removing stopwords...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_marker,args=('code',))
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_clean_lxml)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .


Lemmatization ...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_stopwords)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Body']=df.Body.apply(p5_util.cb_lemmatizer,args=(lemmatizer,'lower'))


In [39]:
if list_tag is not None and list_tag_original is not None and str_original_document is not None: 
    print("\nOriginal sentence : {}".format(str_original_document))
    print("TFIDF/NGRAM=(1,2) : Suggested Tags list : {}".format(list_tag))
    print("Original Tags list : {}\n".format(list_tag_original))


Original sentence : <p>In my user model, I have an attribute called "nickname" and validates as such:</p>

<blockquote>
  <p>validates_format_of :nickname, :with => /[a-zA-Z0-9]$/, :allow_nil => true</p>
</blockquote>

<p>However, it is currently letting this string pass as valid:</p>

<p>a?c</p>

<p>I only want to accept alphanumeric strings - does anyone know why my regular expression is failing?  If anybody could suggest a better regular expression, I'm all ears.</p>

TFIDF/NGRAM=(1,2) : Suggested Tags list : ['allow', 'attribute', 'ci', 'expression', 'pas', 'regular', 'regular expression', 'valid']
Original Tags list : ['regex', 'validation']



In [40]:
import p6_util

#---------------------------------------------------------------
#Extract a row to be tested from test dataset 
#---------------------------------------------------------------
document_index=400
p_tag_ratio=0.3
embeding_mode='bow'

#p6_util.tag_document_test(document_index, embeding_mode, p_tag_ratio)

list_tag, list_tag_original, str_original_document \
= p6_util.taglist_stat_predict(df_sof_test, document_index, embeding_mode, vectorizer, csr_matrix, p_tag_ratio)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeroc words from sentences...

Removing stopwords...

Lemmatization ...


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_remove_marker,args=('code',))
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["Body"] = df.Body.apply(cb_clean_lxml)
/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .


*** ERROR : get_dict_frequency function should apply with         CountVectorizer type only!


/home/bangui/Dropbox/Perso/Formation/Openclassroom/Datascientist/P6/p6_util.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Body']=df.Body.apply(p5_util.cb_lemmatizer,args=(lemmatizer,'lower'))


In [41]:
if list_tag is not None and list_tag_original is not None and str_original_document is not None: 
    print("\nOriginal sentence : {}".format(str_original_document))
    print("TFIDF : Suggested Tags list : {}".format(list_tag))
    print("Original Tags list : {}\n".format(list_tag_original))